<a href="https://colab.research.google.com/github/Pakyy/NLP-Project---MT-en-it/blob/test/seq2seq_pre_addestrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install keras
!pip install tensorflow
!pip install translate-toolkit
!pip install evaluate
!pip install nltk rouge-score
!pip install unbabel-comet;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.2/745.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  P

In [2]:
from google.colab import drive
from translate.storage import tmx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from wordcloud import WordCloud
import tensorflow as tf
import keras
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from tqdm import tqdm
from evaluate import load
import xml.etree.ElementTree as ET
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
from transformers import GenerationConfig

In [3]:
# Monta Google Drive per caricare il file
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/en-it.tmx"
sentence_pairs = []

# Open the file in read mode
with open(file_path, 'rb') as f:
  tmx_file = tmx.tmxfile(f)

# Iterate through the translation units and store sentence pairs
for unit in tmx_file.units:
    source_text = unit.source
    target_text = unit.target
    sentence_pairs.append((source_text, target_text))

# Create a DataFrame from the sentence pairs
df = pd.DataFrame(sentence_pairs, columns=['Source', 'Target']);

Mounted at /content/drive


In [7]:
# Funzione di pulizia per rimuovere caratteri speciali e gestire gli accenti
def clean_text(text):
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"[^a-zA-ZàèéìòùÀÈÉÌÒÙçÇ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Applichiamo la pulizia su Source e Target
df['Source_clean'] = df['Source'].apply(clean_text)
df['Target_clean'] = df['Target'].apply(clean_text)

# Rimuove le righe con frasi vuote
df = df[df['Source_clean'].str.strip() != '']
df = df[df['Target_clean'].str.strip() != '']

# Rimuove frasi troppo corte (esempio: meno di 3 parole)
df = df[df['Source_clean'].apply(lambda x: len(x.split()) >= 3)]
df = df[df['Target_clean'].apply(lambda x: len(x.split()) >= 3)]

# Rimuove frasi troppo lunghe (più di 96 parole)
df = df[df['Source'].apply(lambda x: len(x.split()) <= 96)]
df = df[df['Target'].apply(lambda x: len(x.split()) <= 96)]
# Stampa la lunghezza del dataset
print(f"Dataset length after the removal of too long sentences: {len(df)}")

# Verifica la presenza di eventuali valori nulli
print(df.isnull().sum())

# Rimozione duplicati
df.drop_duplicates(subset=['Source_clean', 'Target_clean'], inplace=True)

# Verifica dei risultati puliti
df[['Source_clean', 'Target_clean']].head(10)

Dataset length after the removal of too long sentences: 153566
Source          0
Target          0
Source_clean    0
Target_clean    0
dtype: int64


,Source_clean,Target_clean
1,There s a tight and surprising link between th...,Esiste uno stretto e sorprendente legame tra l...
2,fish health mission blue oceans science,fish health mission blue oceans science
4,Stephen Palumbi Following the mercury trail,Stephen Palumbi Sulle tracce del mercurio
5,It can be a very complicated thing the ocean,Può essere una cosa davvero complicata l oceano
6,And it can be a very complicated thing what hu...,E può essere una cosa davvero complicata la sa...
7,And bringing those two together might seem a v...,E mettere insieme queste due cose può sembrare...
8,And those simple themes aren t really themes a...,E questi semplici aspetti non riguardano le co...
9,And I m going to start with this one If momma ...,E vorrei partire da questa mamma infelice tutt...
10,We know that right We ve experienced that,Lo sappiamo tutti giusto Ci siamo passati tutti
11,And if we just take that and we build from the...,E se prendiamo questa idea e partiamo da qui p...


In [8]:
# Step 2: Divisione in train e test set e conversione in Hugging Face Dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

# **Specs**

In [23]:
# Imposta i parametri generali
MODEL_NAME = "facebook/bart-base"  # Modello pre-addestrato
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

# **Tokenization**

In [24]:
# Caricamento del tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenizzazione per il modello
def tokenize_data(batch):
    tokenized_input = tokenizer(batch["Source_clean"], padding="max_length", truncation=True, max_length=96)
    tokenized_labels = tokenizer(batch["Target_clean"], padding="max_length", truncation=True, max_length=96)
    tokenized_input["labels"] = tokenized_labels["input_ids"]
    return tokenized_input

train_data = train_data.map(tokenize_data, batched=True)
test_data = test_data.map(tokenize_data, batched=True)

Map:   0%|          | 0/122828 [00:00<?, ? examples/s]

Map:   0%|          | 0/30707 [00:00<?, ? examples/s]

# **Model**

In [16]:
# Step 3: Caricamento del modello e primo test di traduzione
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Set up generation configuration with forced_bos_token_id for Italian
generation_config = GenerationConfig(forced_bos_token_id=tokenizer.convert_tokens_to_ids("<2it>"))

# Primi test di traduzione senza fine-tuning su un campione
sample_texts = test_df['Source_clean'].tolist()[:5]
sample_references = test_df['Target_clean'].tolist()[:5]
inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(inputs["input_ids"])
sample_translations = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Tabella delle traduzioni di esempio
print("\n--- Traduzioni di esempio senza fine-tuning ---")
for i in range(len(sample_texts)):
    print(f"Source: {sample_texts[i]}")
    print(f"Reference: {sample_references[i]}")
    print(f"Model Translation: {sample_translations[i]}\n")


--- Traduzioni di esempio senza fine-tuning ---
Source: We re here to celebrate compassion
Reference: Siamo qui per celebrare la compassione
Model Translation: We re here to celebrate compassion

Source: Well we went to the forests of Singapore and Southeast Asia
Reference: Bene siamo andati nelle foreste di Singapore e del sud est asiatico
Model Translation: Well we went to the forests of Singapore and Southeast Asia

Source: For example they charged Saudi foreign fighters substantially more than Libyans money that would have otherwise gone to al Qaeda
Reference: Per esempio ai combattenti stranieri sauditi addebitavano più che ai libici soldi che altrimenti sarebbero andati ad al Qaeda
Model Translation: For example they charged Saudi foreign fighters substantially more than Libyans money that would have

Source: So when I look at creativity I also think that it is this sense or this inability to repress my looking at associations in practically anything in life
Reference: Allora qu